In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
from __future__ import absolute_import, division, print_function
from builtins import (
    ascii, bytes, chr, dict, filter, hex, input, int, map,
    next, oct, open, pow, range, round, str, super, zip)

# Standard library imports
import os
from functools import partial
from math import pi
import json
from collections import defaultdict
import random

import numpy as np
import pandas as pd
import networkx as nx

# Imports for working with shapefiles
import pyproj
from shapely.geometry import (
    shape,
    MultiPolygon,
    mapping
)
from shapely.ops import (
    transform,
    cascaded_union
)
import fiona
from fiona.crs import from_epsg

# local imports
from src.modelling.input import shapes_to_graph
from src.modelling.clustering import SSGraphKMeans

In [4]:
%matplotlib inline

In [5]:
# Create pandas dataframes that have information about each blockgroup
poptot_df = pd.read_csv('data/block_groups/pop_tot/DEC_10_SF1_P1_with_ann.csv')
poptot_df = poptot_df[['GEO.id2', 'D001']]
poptot_df.columns = ['geoid', 'poptot']
poptot_df.drop(0, axis=0, inplace=True)
poptot_df.set_index('geoid', inplace=True)
poptot_df['poptot'] = poptot_df['poptot'].apply(int)

In [6]:
wisc_census_blocks = 'data/block_groups/shapes/tl_2013_55_bg.shp'

# A convenience object for projecting lat/long values
# from EPSG 4326 to 3695 (approximate xy mappings for
# central Wisconsin)
project = partial(
    pyproj.transform,
    pyproj.Proj(init='epsg:4326'), 
    pyproj.Proj(init='epsg:3695')
)

In [7]:
# Create a list of blockgroups, which has a shape, a geoid,
# and an untransformed shape
with fiona.open(wisc_census_blocks) as f:
    blocks = [
        {
            'shape': shape(block['geometry']), 
            'geoid': block['properties']['GEOID']
        }
        for block in f
    ]

In [8]:
blocks_graph = shapes_to_graph(blocks)

In [9]:
block_geoids = [
    block['geoid']
    for block in blocks
]

In [10]:
block_populations = {
    geoid: poptot_df.to_dict()['poptot'][geoid]
    for geoid in block_geoids
}

In [23]:
graph_cluster = SSGraphKMeans()

In [24]:
graph_cluster.fit(blocks_graph, block_populations)

Annealing cluster weights to within 50000.0
Annealing cluster 6
Calculating metrics of the graph not in cluster 6
This will take some time.
Finished calculations. Continuing.
Annealing cluster 7
Calculating metrics of the graph not in cluster 7
This will take some time.
Finished calculations. Continuing.
Reconnecting cluster 5
Reconnecting cluster 5
Annealing cluster 1
Calculating metrics of the graph not in cluster 1
This will take some time.
Finished calculations. Continuing.
Annealing cluster 5
Calculating metrics of the graph not in cluster 5
This will take some time.
Finished calculations. Continuing.
Reconnecting cluster 2
Reconnecting cluster 4
Reconnecting cluster 5
Annealing cluster 8
Annealing cluster 4
Calculating metrics of the graph not in cluster 4
This will take some time.
Finished calculations. Continuing.
Reconnecting cluster 2
Reconnecting cluster 2
Reconnecting cluster 2
Reconnecting cluster 4
Reconnecting cluster 4
Reconnecting cluster 4
Calculating metrics of the g

KeyboardInterrupt: 

In [21]:
for i in range(1, 9):
    print(i, nx.is_connected(blocks_graph.subgraph(graph_cluster.clusters[i])))

1 True
2 True
3 True
4 True
5 True
6 True
7 True
8 True


In [22]:
graph_cluster.cluster_weights

{1: 686204,
 2: 378758,
 3: 699405,
 4: 788029,
 5: 575456,
 6: 711620,
 7: 1123123,
 8: 724391}

In [ ]:
'550759610002' in graph_cluster.clusters[6].border

In [ ]:
graph_cluster.clusters.keys()

In [ ]:
members_border_totals = [
    (len(graph_cluster.clusters[i]), len(graph_cluster.clusters[i].border))
    for i in range(1, 9)
]

In [ ]:
members_border_totals

In [ ]:
foo = list(nx.connected_component_subgraphs(blocks_graph.subgraph(graph_cluster.clusters[7])))

In [ ]:
foo[1].nodes()